## Library imports

In [1]:
import numpy as np
import pandas as pd
from chromatography import ExperimentAnalytes
from separation_utility import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import time
%matplotlib qt

## Dataset Import

In [2]:
alists = []
alists.append(pd.read_csv(f'../data/GilarSample.csv'))
alists.append(pd.read_csv(f'../data/Alizarin.csv'))
alists.append(pd.read_csv(f'../data/Peterpeptides.csv'))
alists.append(pd.read_csv(f'../data/Roca.csv'))
alists.append(pd.read_csv(f'../data/Peter32.csv'))
alists.append(pd.read_csv(f'../data/Eosin.csv'))
alists.append(pd.read_csv(f'../data/Controlmix2.csv'))
# GilarSample - 8 analytes
# Peterpeptides - 32 analytes
# Roca - 14 analytes
# Peter32 - 32 analytes
# Eosin - 20 analytes
# Alizarin - 16 analytes
# Controlmix2 - 17 analytes
# Gooding - 872 analytes
#alist['k0'] = np.exp(alist.lnk0)

## Useful functions definition

In [3]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses
def average_over_equal_intervals(arr, interval):
    return np.mean(arr.reshape(-1, interval), axis=1)

In [ ]:
# Parameters
sample = 'Peterpeptides'
alist = pd.read_csv(f'../data/{sample}.csv')
# GilarSample
run_time_lim = 10.0
sigma_max = 0.3
delta_taus = [.25, .25, 10]
num_episodes = 5000
sample_size = 10
lr = .01
optim = lambda a, b: torch.optim.SGD(a, b)#, momentum=0.65)
lr_decay_factor = .5
lr_milestones = 1000
print_every= 100
baseline = 0.55
max_norm = 2.
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, run_time=10.0)
losses_dist = []

start_process_peterp = time.perf_counter()

for i in range(50):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus= delta_taus, 
            num_episodes=num_episodes, 
            sample_size=sample_size,
            optim=optim,
            lr=lr, 
            print_every=print_every,
            lr_decay_factor=lr_decay_factor,
            lr_milestones=lr_milestones,
            baseline=baseline,
            max_norm=max_norm,

        )
    exp.reset()
    exp.run_all(pol.mu.detach().numpy(), delta_taus)
    
    losses_dist.append(exp.loss())
end_process_peterp = time.perf_counter()
# time 1.8 min

In [ ]:
(end_process_peterp - start_process_peterp)/3000

In [ ]:
plt.hist(losses_dist, bins=50)
plt.title(f"Final Result Distribution for SingleSetModel (PeterPeptides)")
plt.xlabel("Loss")
plt.ylabel("Occurrences")

In [ ]:
# Parameters
sample = 'GilarSample'
alist = pd.read_csv(f'../data/{sample}.csv')
# GilarSample
run_time_lim = 10.0
sigma_max = 0.3
delta_taus = [.25, .25, 10]
num_episodes = 5000
sample_size = 10
lr = .01
optim = lambda a, b: torch.optim.SGD(a, b)#, momentum=0.65)
lr_decay_factor = .5
lr_milestones = 1000
print_every= 100
baseline = 0.55
max_norm = 2.
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, run_time=10.0)
losses_dist_gil = []

start_process_gil = time.perf_counter()

for i in range(50):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus= delta_taus, 
            num_episodes=num_episodes, 
            sample_size=sample_size,
            optim=optim,
            lr=lr, 
            print_every=print_every,
            lr_decay_factor=lr_decay_factor,
            lr_milestones=lr_milestones,
            baseline=baseline,
            max_norm=max_norm,

        )
    exp.reset()
    exp.run_all(pol.mu.detach().numpy(), delta_taus)
    
    losses_dist_gil.append(exp.loss())
end_process_gil = time.perf_counter()

In [ ]:
plt.hist(losses_dist_gil, bins=50)
plt.title(f"Final Result Distribution for SingleSetModel (GillarSample)")
plt.xlabel("Loss")
plt.ylabel("Occurrences")

### Experiment: Performance vs n_steps

In [ ]:
def my_(ab = 10, cd = "shit")

In [12]:
# Parameters
sigma_max = 0.3

kwargs = {
'num_episodes' : 6000,
'sample_size' : 10,
'lr' : .05,
'optim' : torch.optim.SGD,
'lr_decay_factor' : .5,
'lr_milestones' : 1000,
'print_every' : 6001,
'baseline' : 0.55,
'max_norm' : 2.
}
N = 7
M = 100
# Experiments
exp_8 = ExperimentAnalytes(k0 = alists[0].k0.values, S = alists[0].S.values, h=0.001, run_time=1.0)
exp_16 = ExperimentAnalytes(k0 = alists[1].k0.values, S = alists[1].S.values, h=0.001, run_time=1.0)
exp_32 = ExperimentAnalytes(k0 = alists[2].k0.values, S = alists[2].S.values, h=0.001, run_time=1.0)
# Final Results 
dist_8 = np.zeros((N, M))
dist_16 = np.zeros((N, M))
dist_32 = np.zeros((N, M))

for n in range(0, N):
    print(n)
    delta_taus = np.linspace(0, 1, n + 3)[1:]
    
    for i in range(M):
        #Policies
        pol_8 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        pol_16 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        pol_32 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        # Run Exp 8
        reinforce_one_set(
                exp_8, 
                pol_8, 
                delta_taus=delta_taus, 
                **kwargs
            )
        # Run Exp 16
        reinforce_one_set(
                exp_16, 
                pol_16, 
                delta_taus=delta_taus, 
                **kwargs
            )
        # Run Exp 32
        reinforce_one_set(
                exp_32, 
                pol_32, 
                delta_taus=delta_taus, 
                **kwargs
            )
        
        exp_8.reset()
        exp_16.reset()
        exp_32.reset()
        mu_8, _ = pol_8.forward()
        mu_16, _ = pol_16.forward()
        mu_32, _ = pol_32.forward()
        exp_8.run_all(mu_8.detach().numpy(), delta_taus)
        exp_16.run_all(mu_16.detach().numpy(), delta_taus)
        exp_32.run_all(mu_32.detach().numpy(), delta_taus)
        
        dist_8[n, i] = exp_8.loss()
        dist_16[n, i] = exp_16.loss()
        dist_32[n, i] = exp_32.loss()




0


/home/stefan/Thesis/code/chromatography.py:250: RuntimeWarning: divide by zero encountered in true_divide
  return delta_tau_phi * (1 + self.k(phi)) / self.k(phi)


1


/home/stefan/Thesis/code/chromatography.py:250: RuntimeWarning: overflow encountered in true_divide
  return delta_tau_phi * (1 + self.k(phi)) / self.k(phi)


2


KeyboardInterrupt: 

In [ ]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), delta_taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.show()
plt.legend()
#plt.savefig(f'results/{sample}_result_{run}.png')

In [ ]:
exp.reset()
exp.run_all(best_program, delta_taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.legend()
#plt.savefig(f'results/{sample}_best_result_{run}.png')

In [ ]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
#plt.plot(mus[:, 2], label='Mu: phi3')
#plt.plot(mus[:, 3], label='Mu: phi4')
plt.ylim((0,1))
plt.legend()
#plt.savefig(f'results/{sample}_mus_{run}.png')

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
#plt.plot(sigmas[:, 2], label='Sigma: phi3')
#plt.plot(sigmas[:, 3], label='Sigma: phi4')
plt.ylim((0,sigma_max))
plt.legend()
#plt.savefig(f'results/{sample}_sigmas_{run}.png')

In [ ]:
plt.plot(losses)
plt.xlabel("Episodes")
plt.ylabel("Loss")
plt.title("Loss during learning process")
#plt.savefig(f'results/{sample}_loss_{run}.png')

In [ ]:
X, Y, Loss_field = loss_field(exp, [.25, 30000], N = 500)
plt.contourf(X, Y, Loss_field, levels=100)
plt.xlabel("Phi1")
plt.ylabel("Phi2")
index = np.unravel_index(np.argmin(Loss_field), Loss_field.shape)
plt.scatter(X[index], Y[index], c="r", s=5)
plt.title(f"Loss Field, change of phi at 0.5\nGlobal min ({round(np.min(Loss_field), 3)}) at {round(X[index], 4), round(Y[index], 4)}, red dot")
#plt.savefig(f'results/{sample}_loss_filed_at_5.png')

In [ ]:
N = -1
mlab.surf(X.T[:N, :N], Y.T[:N, :N], Loss_field.T[:N, :N])

In [ ]:

phis = np.linspace(0, 1, 1000)
losses = []
for phi1 in phis:
    exp.reset()
    exp.run_all([phi1], [10])
    losses.append(exp.loss())
plt.plot(losses[:])
plt.ylim((0, 2))

In [ ]:
plt.contourf(X, Y, Loss_field, levels=100)

In [ ]:
exp_time = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol_time = PolicyTime(2, sigma_max = .2)
losses_time, best_program_time, mus_time, sigmas_time, n_par_time = reinforce_delta_tau(
        exp_time, 
        pol_time,
        num_episodes=5000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
        lr=.1, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=0.5),
        baseline=0.65,
        max_norm=2.
    )

In [ ]:
exp_time.reset()
exp_time.run_all(pol_time.mu.detach().numpy()[:2], [0.25, 3000000])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
exp_time.reset()
exp_time.run_all(best_program_time[0], best_program_time[1])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
plt.plot(losses_time)

In [ ]:
plt.plot(mus_time[:, 0], label='Mu: 0')
plt.plot(mus_time[:, 1], label='Mu: 1')
plt.plot(mus_time[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas_time[:, 0], label='Sigma: 0')
plt.plot(sigmas_time[:, 1], label='Sigma: 1')
plt.plot(sigmas_time[:, 2], label='Sigma: 2')
plt.plot(sigmas_time[:, 3], label='Sigma: 3')
plt.plot(sigmas_time[:, 4], label='Sigma: 4')
plt.plot(sigmas_time[:, 5], label='Sigma: 5')
plt.plot(sigmas_time[:, 6], label='Sigma: 6')

plt.legend()
plt.show()